In [ ]:
#!pip install ipykernel
#!pip install pickle-mixin
#!pip install argparse

In [13]:
import json
import logging
import sys
import plac
import argparse
import os
import pickle

Convert .tsv file to dataturks json format. 


In [14]:
def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'r') # input file
        print(input_path)
        fp=open(output_path, 'w') # output file
        print(output_path)
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        cuerpo=False
        for line in f:
            if not cuerpo:
                cuerpo=True
                continue 
            print(line)
            #if line[0:len(line)-1]!='.\tO':
            if line[0]!='.':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                print("punto")
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                print(annotations)
                annotations=[]
                json.dump(data_dict, fp, ensure_ascii=False)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None

tsv_to_json_format('NLP.tsv','NLP.json','')
print("listo")

NLP.tsv
NLP.json
PUBLIC DEED OF SALE	DOC

.	0

punto
[{'label': ['DOC'], 'points': [{'text': 'PUBLIC DEED OF SALE', 'start': 0, 'end': 18}]}]
NUMBER	0

THOUSAND THREE HUNDRED FIFTEEN	CARDINAL

(	0

1335	CARDINAL

)	0

.	0

punto
[{'label': ['CARDINAL'], 'points': [{'text': 'THOUSAND THREE HUNDRED FIFTEEN', 'start': 7, 'end': 36}]}, {'label': ['CARDINAL'], 'points': [{'text': '1335', 'start': 40, 'end': 43}]}]
In	0

Alicante	GPE

,	0

on	0

the	0

fourteenth of October	TIME

,	0

nineteen ninety-seven	TIME

.	0

punto
[{'label': ['GPE'], 'points': [{'text': 'Alicante', 'start': 3, 'end': 10}]}, {'label': ['TIME'], 'points': [{'text': 'fourteenth of October', 'start': 21, 'end': 41}]}, {'label': ['TIME'], 'points': [{'text': 'nineteen ninety-seven', 'start': 45, 'end': 65}]}]
Before	0

me	0

,	0

RAFAEL MUÑOZ GUILLENA	PERSON

,	0

Notary Public	PROFESION

of	0

the	0

Illustrious College of Alicante	ORG

,	0

resident	0

in	0

Playa de San Juan	LOC

,	0

district	0

of	0

Alicante	GPE

,

Convert json file to spaCy format.

In [15]:
@plac.annotations(input_file=("Input file", "option", "i", str), output_file=("Output file", "option", "o", str))

def to_pickle(input_file='NLP.json', output_file='NLP.pickle'):
    try:
        training_data = []
        lines=[]
        with open(input_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        print(training_data)

        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

    except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))
        return None

to_pickle(input_file='NLP.json', output_file='NLP.pickle')

[('PUBLIC DEED OF SALE ', {'entities': [(0, 19, 'DOC')]}), ('NUMBER THOUSAND THREE HUNDRED FIFTEEN ( 1335 ) ', {'entities': [(7, 37, 'CARDINAL'), (40, 44, 'CARDINAL')]}), ('In Alicante , on the fourteenth of October , nineteen ninety-seven ', {'entities': [(3, 11, 'GPE'), (21, 42, 'TIME'), (45, 66, 'TIME')]}), ('Before me , RAFAEL MUÑOZ GUILLENA , Notary Public of the Illustrious College of Alicante , resident in Playa de San Juan , district of Alicante , APPEARS : The spouses DON ISMAEL GARCIA NAVARRO and DOÑA ISIDORA MARTINEZ MORENO , of legal age , mechanic he and manager her , neighbors of Montornès del Vallés , Castell street, number 3 , with DDNNII-NNIIFF 27,322,177-M and 51,347,341-P , respectively ', {'entities': [(12, 33, 'PERSON'), (172, 193, 'PERSON'), (203, 226, 'PERSON'), (36, 49, 'PROFESION'), (244, 252, 'PROFESION'), (260, 267, 'PROFESION'), (57, 88, 'ORG'), (103, 120, 'LOC'), (287, 307, 'LOC'), (310, 334, 'LOC'), (135, 143, 'GPE'), (342, 355, 'DOC'), (356, 368, 'DNI'), 